In [20]:
from requests import get
from json import loads
from pandas import DataFrame, concat
from datetime import datetime    

In [ ]:
  

regular_season = get('https://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/' + str(season) + '/types/2/events?limit=1000')
regular_season = loads(regular_season.content.decode('utf-8'))
regular_season = DataFrame.from_dict(regular_season.get('items'))
regular_season = regular_season.rename(columns={"$ref": "url"})
regular_season['game_id'] = regular_season['url'].apply(lambda x: x.replace('http://sports.core.api.espn.com/v2/sports/football/leagues/college-football/events/', '').replace('?lang=en&region=us', ''))
regular_season['season_type'] = 'regular'

In [29]:
def get_schedule(season = datetime.now().year - 1, season_type = 2):

    # Arg Validation
    if season_type in [2, 3]:
        ValueError("season type must be 2 for regular season or 3 for post season")

    if type(season) != int:
        ValueError('season must be int')

    # Future args if number of games in a season exceeds 1000
    API_MAX_PAGE_SIZE = 1000
    PAGE = 1

    # body
    request_url = 'https://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/{}/types/{}/events?limit={}&page={}'.format(season, season_type, API_MAX_PAGE_SIZE, PAGE)
    schedule = get(request_url)
    schedule = loads(schedule.content.decode('utf-8'))
    schedule = DataFrame.from_dict(schedule.get('items'))
    schedule = schedule.rename(columns={"$ref": "url"})
    schedule['game_id'] = schedule['url'].apply(lambda x: x.replace('http://sports.core.api.espn.com/v2/sports/football/leagues/college-football/events/', '').replace('?lang=en&region=us', ''))
    schedule['season_type'] = 'regular'

    return schedule

In [31]:
season = get_schedule()
season

,url,game_id,season_type
0,http://sports.core.api.espn.com/v2/sports/foot...,401523986,regular
1,http://sports.core.api.espn.com/v2/sports/foot...,401525434,regular
2,http://sports.core.api.espn.com/v2/sports/foot...,401520145,regular
3,http://sports.core.api.espn.com/v2/sports/foot...,401525450,regular
4,http://sports.core.api.espn.com/v2/sports/foot...,401532392,regular
...,...,...,...
871,http://sports.core.api.espn.com/v2/sports/foot...,401539477,regular
872,http://sports.core.api.espn.com/v2/sports/foot...,401539481,regular
873,http://sports.core.api.espn.com/v2/sports/foot...,401539482,regular
874,http://sports.core.api.espn.com/v2/sports/foot...,401539484,regular


In [32]:
def get_game(game_id):
    res = get('http://site.api.espn.com/apis/site/v2/sports/football/college-football/summary?event=' + game_id)
    return res.content.decode('utf-8')

In [34]:
get_game(season['game_id'].values[0])

'{"boxscore":{"teams":[{"team":{"id":"23","uid":"s:20~l:23~t:23","slug":"san-jose-state-spartans","location":"San José State","name":"Spartans","abbreviation":"SJSU","displayName":"San José State Spartans","shortDisplayName":"San José St","color":"005893","logo":"https://a.espncdn.com/i/teamlogos/ncaa/500/23.png"},"statistics":[{"name":"firstDowns","displayValue":"23","label":"1st Downs"},{"name":"thirdDownEff","displayValue":"5-13","label":"3rd down efficiency"},{"name":"fourthDownEff","displayValue":"0-1","label":"4th down efficiency"},{"name":"totalYards","displayValue":"396","label":"Total Yards"},{"name":"netPassingYards","displayValue":"198","label":"Passing"},{"name":"completionAttempts","displayValue":"21-39","label":"Comp-Att"},{"name":"yardsPerPass","displayValue":"5.1","label":"Yards per pass"},{"name":"rushingYards","displayValue":"198","label":"Rushing"},{"name":"rushingAttempts","displayValue":"27","label":"Rushing Attempts"},{"name":"yardsPerRushAttempt","displayValue":"